In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'Chopped-10-Years-of-Episode-Data'
device = 'cuda'

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('$100')

['$', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['how','hi','how'])

array([0., 1., 0.])

In [10]:
data = pd.read_csv('./data.csv').dropna()

In [11]:
X = data['episode_notes']

In [13]:
y = data['entree']

In [14]:
data

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,entree,dessert,contestant1,contestant1_info,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info
0,1,1,1,"""Octopus, Duck, Animal Crackers""",This is the first episode with only three offi...,"January 13, 2009",Marc Murphy,Alex Guarnaschelli,Aarón Sánchez,"baby octopus, bok choy, oyster sauce, smoked ...","duck breast, green onions, ginger, honey","prunes, animal crackers, cream cheese",Summer Kriegshauser,Private Chef and Nutrition Coach New York NY,Perry Pollaci,Private Chef and Sous chef Bar Blanc New Yo...,Katie Rosenhouse,Pastry Chef Olana Restaurant New York NY,Sandy Davis,Catering Chef Showstoppers Catering at Union...
1,1,2,2,"""Tofu, Blueberries, Oysters""",This is the first of a few episodes with five ...,"January 20, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"firm tofu, tomato paste, prosciutto","daikon, pork loin, Napa cabbage, Thai chiles,...","phyllo dough, gorgonzola cheese, pineapple ri...",Raymond Jackson,Private Caterer and Culinary Instructor West...,Klaus Kronsteiner,Chef de cuisine Liberty National Golf Course...,Christopher Jackson,Executive Chef and Owner Ted and Honey Broo...,Pippa Calland,Owner and Chef Chef for Hire LLC Newville PA
3,1,4,4,"""Banana, Collard Greens, Grits""","In the appetizer round, Chef Chuboda refused t...","February 3, 2009",Scott Conant,Amanda Freitag,Geoffrey Zakarian,"ground beef, wonton wrappers, cream of mushro...","scallops, collard greens, anchovies, sour cream","maple syrup, black plums, almond butter, waln...",Sean Chudoba,Executive Chef Ayza Wine Bar New York NY,Kyle Shadix,Chef Registered Dietician and Culinary Consu...,Luis Gonzales,Executive Chef Knickerbocker Bar & Grill Ne...,Einat Admony,Chef and Owner Taïm New York NY
6,1,7,7,"""Quail, Arctic Char, Beer""",In the appetizer Melinda did not get her quail...,"February 24, 2009",Scott Conant,Alex Guarnaschelli,Amanda Freita,"endive, frozen peas, quail, red seedless grapes","watercress, tamarind paste, creamed corn, Arc...","pitted dates, frozen pie crust, caramel candy...",Melinda Dorn,Sous chef and Instructor New York NY,Ralph Battista Jr.,Executive Chef Butterfield 8 New York NY,Lior Lev Sercarz,Chef and Entrepreneur La Boîte à Biscuits an...,Abe Lopez,Chef de cuisine Pacifico Center Valley PA
7,1,8,8,"""Coconut, Calamari, Donuts""","In the appetizer round, Chef LePape failed to ...","March 3, 2009",Aarón Sánchez,Alex Guarnaschelli,Chris Santo,"turbot, nori sheets, pistachios, coconut","rainbow chard, papaya, granola, parsnips, cal...","plain donuts, smoked gouda, white chocolate, ...",Gwen LePape,Executive Chef Frederick's Restaurant New Y...,Francesco Peluso,Executive Chef Peluso's Italian Specialties ...,Dave Ogren,Executive Chef Almos Restaurant New York NY,Mina Newman,Executive Chef Christos Steakhouse Astoria ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,45,8,562,"""Soda Flop""",The fish heads in the first basket were from s...,"June 16, 2020",Marc Murphy,Amanda Frietag,David Burtka,"fish heads, pea shoots, white asparagus, chic...","baked cacio e pepe, rock shrimp, mustard gree...","peaches, rosemary, pretzel shortbread, chicke...",Jason Hanelt,Executive Chef from Boston MA,David Sierra,Executive Chef from Washington D.C.,Matt McPherson,Chef and Owner from Boston MA,Maame Boakye,Private Chef from New York NY
564,45,9,563,"""Terrine Cuisine""",Chef Jose cut himself in the first round and c...,"June 23, 2020",Chris Santos,Scott Conant,Erik Ramirez,"rabbit terrine, guanciale, spring garlic, bur...","beer ramen, sablefish, corn on the cob, lemon...","feta ice cream, pears, blueberry ketchup, cho...",Jose Luis Chavez,Chef and Owner from New York NY,Matt Greiner,Executive Chef from Raleigh NC,Mimi Weissenborn,Executive Chef from Harlem NY,Nemo Bolin,Chef and Owner from Providence RI
565,45,10,564,"""Time and Turmoil""",Chef A